In [1]:
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [7]:
df = pd.read_csv('Boston Marathon 2014.csv')

In [8]:
df.head()

,10k,name,division,25k,gender,age,official,bib,genderdiv,ctz,...,overall,pace,state,30k,5k,half,20k,country,city,40k
0,17.37,"Yamamoto, Hiroyuki",8,47.67,M,47,85.25,W1,8,NaN,...,8,3.27,NaN,59.18,8.02,39.72,37.65,JPN,Fukuoka,80.43
1,32.58,"Jeptoo, Rita",1,82.43,F,33,138.95,F1,1,NaN,...,21,5.30,NaN,99.33,16.22,69.47,65.83,KEN,Eldoret,132.10
2,16.62,"Van Dyk, Ernst F.",1,45.80,M,41,80.60,W2,1,NaN,...,1,3.08,NaN,56.45,7.75,38.03,36.10,RSA,Paarl,76.10
3,32.57,"Dibaba, Mare",3,82.43,F,24,140.58,F2,3,NaN,...,27,5.37,NaN,99.33,16.20,69.47,65.83,ETH,Shoa,132.95
4,17.12,"Hokinoue, Kota",2,46.37,M,40,81.23,W3,2,NaN,...,2,3.10,NaN,57.03,8.02,38.60,36.58,JPN,Nogata Fukuoka,76.72


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31984 entries, 0 to 31983
Data columns (total 21 columns):
10k          31984 non-null object
name         31984 non-null object
division     31984 non-null int64
25k          31984 non-null object
gender       31984 non-null object
age          31984 non-null int64
official     31984 non-null float64
bib          31984 non-null object
genderdiv    31984 non-null int64
ctz          1244 non-null object
35k          31984 non-null object
overall      31984 non-null int64
pace         31984 non-null float64
state        29408 non-null object
30k          31984 non-null object
5k           31984 non-null object
half         31984 non-null object
20k          31984 non-null object
country      31984 non-null object
city         31983 non-null object
40k          31984 non-null object
dtypes: float64(2), int64(4), object(15)
memory usage: 5.1+ MB


In [11]:
df['ctz'].describe()

count     1244
unique      84
top        GBR
freq       171
Name: ctz, dtype: object

In [13]:
df.drop('ctz', axis=1, inplace=True)

In [21]:
df['10k'] = pd.to_numeric(df['10k'], errors='coerce')
df['25k'] = pd.to_numeric(df['25k'], errors='coerce')
df['35k'] = pd.to_numeric(df['35k'], errors='coerce')
df['30k'] = pd.to_numeric(df['30k'], errors='coerce')
df['5k'] = pd.to_numeric(df['5k'], errors='coerce')
df['20k'] = pd.to_numeric(df['20k'], errors='coerce')
df['40k'] = pd.to_numeric(df['40k'], errors='coerce')
df['half'] = pd.to_numeric(df['half'], errors='coerce')


In [22]:
df['state'].isnull().sum()

2576

In [30]:
from sklearn.preprocessing import LabelEncoder
lb_maker = LabelEncoder()

In [28]:
df.dropna(inplace=True)

In [32]:
df['gender'] = lb_maker.fit_transform(df['gender'])
df['bib'] = lb_maker.fit_transform(df['bib'])
df['state'] = lb_maker.fit_transform(df['state'])
df['country'] = lb_maker.fit_transform(df['country'])
df['city'] = lb_maker.fit_transform(df['city'])

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29103 entries, 9 to 31983
Data columns (total 21 columns):
10k          29103 non-null float64
name         29103 non-null object
division     29103 non-null int64
25k          29103 non-null float64
gender       29103 non-null int64
age          29103 non-null int64
official     29103 non-null float64
bib          29103 non-null int64
genderdiv    29103 non-null int64
35k          29103 non-null float64
overall      29103 non-null int64
pace         29103 non-null float64
state        29103 non-null int64
30k          29103 non-null float64
5k           29103 non-null float64
half         29103 non-null float64
20k          29103 non-null float64
country      29103 non-null int64
city         29103 non-null int64
40k          29103 non-null float64
25           29103 non-null float64
dtypes: float64(11), int64(9), object(1)
memory usage: 4.9+ MB


Data is clean, on to modeling

# Modeling

In [35]:
X = df.drop(['name','gender'], axis=1)

In [36]:
y = df['gender']

In [37]:
from sklearn.preprocessing import normalize

In [38]:
X_norm = normalize(X)

In [40]:
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score

In [41]:
i = 0
results = []
for i in range(1,20):
    y_pred = KMeans(n_clusters=i, random_state=101).fit_predict(X_norm)
    results.append(accuracy_score(y, y_pred))
    i+=1

In [42]:
results

[0.4647287221248669,
 0.6518228361337319,
 0.2386008315293956,
 0.2684946569082225,
 0.46692780812974605,
 0.3093839123114456,
 0.2701783321307082,
 0.17389959797958973,
 0.17211284060062537,
 0.11813215132460571,
 0.007902965330034704,
 0.2056489021750335,
 0.009311754801910457,
 0.17383087654193727,
 0.17743875201869222,
 0.08975019757413324,
 0.032127272102532384,
 0.1301927636326152,
 0.1252791808404632]

Best accuracy achieved with two clusters, which makes sense since we are trying to predict gender

In [44]:
from sklearn.decomposition import PCA

In [45]:
X_pca = PCA(2).fit_transform(X_norm)

In [46]:
y_pred = KMeans(n_clusters=2, random_state=101).fit_predict(X_pca)

In [47]:
print(accuracy_score(y, y_pred))

0.6517884754149057
